### Model Evaluation

In [1]:
import os

In [2]:
%pwd

'd:\\workspace-mlops\\MLOps-Project-02-End-to-End-Data-Science-Project-with-Deployment\\research'

In [3]:
os.chdir('../')
%pwd

'd:\\workspace-mlops\\MLOps-Project-02-End-to-End-Data-Science-Project-with-Deployment'

In [4]:
%ls

 Volume in drive D is Personal's Data
 Volume Serial Number is 9416-D6A6

 Directory of d:\workspace-mlops\MLOps-Project-02-End-to-End-Data-Science-Project-with-Deployment

11/25/2024  07:02 PM    <DIR>          .
11/24/2024  05:51 PM    <DIR>          ..
11/26/2024  11:03 AM               232 .env
11/26/2024  11:02 AM               118 .env.example
11/24/2024  09:21 AM    <DIR>          .github
11/25/2024  09:41 AM                42 .gitignore
11/25/2024  07:02 PM    <DIR>          artifacts
11/24/2024  09:21 AM    <DIR>          config
11/24/2024  09:21 AM                 0 Dockerfile
11/25/2024  09:40 AM    <DIR>          logs
11/25/2024  06:59 PM             2,461 main.py
11/25/2024  06:21 PM                42 params.yaml
11/26/2024  10:55 AM             3,541 README.md
11/24/2024  08:55 AM               134 requirements.txt
11/26/2024  10:59 AM    <DIR>          research
11/25/2024  10:17 AM               334 schema.yaml
11/24/2024  11:20 AM                69 setup.py
11/24/2024  

In [6]:
# reading .env file and setting environment variables
from dotenv import load_dotenv
load_dotenv()


True

In [8]:
# print(os.environ['MLFLOW_TRACKING_URI'])
# print(os.environ['MLFLOW_TRACKING_USERNAME'])
# print(os.environ['MLFLOW_TRACKING_PASSWORD'])


In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [11]:
from src.datasience.constants import *
from src.datasience.utils.common import read_yaml, create_directories, save_json

In [12]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            all_params = params,
            target_column = schema.name,
            mlflow_uri = os.environ['MLFLOW_TRACKING_URI']
        )

        return model_evaluation_config

In [13]:
import os
import pandas as pd
import numpy as np
import mlflow 
import mlflow.sklearn
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse


In [14]:

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model" )

In [15]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config = model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-11-26 11:31:45,546: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-26 11:31:45,553: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-26 11:31:45,558: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-26 11:31:45,564: INFO: common: created directory at: artifacts]
[2024-11-26 11:31:45,564: INFO: common: created directory at: artifacts/model_evaluation]
[2024-11-26 11:31:47,340: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2024/11/26 11:31:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticnetModel'.
2024/11/26 11:32:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.


🏃 View run likeable-swan-434 at: https://dagshub.com/asadhanif3188/MLOps-Project-02-End-to-End-Data-Science-Project-with-Deployment.mlflow/#/experiments/0/runs/8bb2f40af0cd4383a5a8067ebd295a3e
🧪 View experiment at: https://dagshub.com/asadhanif3188/MLOps-Project-02-End-to-End-Data-Science-Project-with-Deployment.mlflow/#/experiments/0
